In [1]:
import os
import glob
import datetime
import pandas as pd
import numpy as np
import xarray as xr
#import WAPORWA modules
os.chdir(r'D:\Github\WAPORWA\modules') #change to modules path
import WA
from WA.pickle_basin import pickle_in,pickle_out  
from WA.model_SMBalance import open_nc
from WA import GIS_functions as gis
from dask.distributed import Client

#Read pickle
Main_dir=r"D:\Github\WAPORWA\data\Litani\Main"
pickle=glob.glob(os.path.join(Main_dir,'*.pickle'))[-1] 
BASIN=pickle_in(pickle)  

#Customize dask performance
client = Client(processes=False, 
                threads_per_worker=2,
                n_workers=2, 
                memory_limit='4GB')
client

C:\ProgramData\Anaconda3\lib\site-packages\dask\config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
C:\ProgramData\Anaconda3\lib\site-packages\distributed\config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


Client Scheduler: inproc://192.168.2.9/1956/1 Dashboard: http://localhost:8787/status,Cluster Workers: 2 Cores: 4 Memory: 8.00 GB


# Aggregate annual value by hydrological year


In [2]:
time='A-{0}'.format(BASIN['end_month'])
for key in ['p','et','etincr','etrain']:
    nc=BASIN['main_data']['monthly'][key]
    var,name=open_nc(nc)
    var_y=var.resample(time=time).sum(dim='time',skipna=False)
    outfolder=os.path.join(BASIN['Dir'],'data','nc')  
    attrs={"units":"mm/year", "source": "Hydrolological year ", "quantity":name}
    var_y.assign_attrs(attrs)
    var_y.name = name
    var_y_dts=var_y.chunk({"latitude":-1,"longitude":-1}).to_dataset()
    nc_fn="{0}_hyearly.nc".format(key)
    nc_path=os.path.join(outfolder,nc_fn)
    var_y_dts.to_netcdf(nc_path,encoding={name:{'zlib':True}})
    BASIN['main_data']['yearly'][key]=nc_path
pickle_out(BASIN)

'D:\\Github\\WAPORWA\\data\\Litani\\Main\\Info_20200717_16h38.pickle'

# Calculate yearly fluxes volumes table for Sheet 1

In [3]:
#%% Create Area mask
# get template from LU map
template=glob.glob(os.path.join(BASIN['input_data']['yearly']['lu'][0],"*.tif"))[0]
driver, NDV, xsize, ysize, GeoT, Projection = gis.GetGeoInfo(template)
# calculate area per pixel
area_map=gis.MapPixelAreakm(template)
# save area map as tif
BASIN['input_data']['stat']['area']=os.path.join(BASIN['Dir'],'data','stat','Area_km.tif')
gis.CreateGeoTiff(BASIN['input_data']['stat']['area'],area_map,driver, NDV, xsize, ysize, GeoT, Projection)

# create area mask
shape=BASIN['geo_data']['basin'] #Shapefile of the area of interest
area=BASIN['input_data']['stat']['area']
BASIN['main_data']['stat']['area']=os.path.join(BASIN['Dir'],'data','stat','Basin_Area_km.tif')
gis.Clip_shapefile(area,shape,BASIN['main_data']['stat']['area'])

In [4]:
### Select area of interest
area_fh=BASIN['main_data']['stat']['area'] # The whole Basin area
# area_fh=Basin['main_data']['subbasin']['Zarqa'] #or sub-basin area
area=gis.OpenAsArray(area_fh,nan_values=True)

### Calculate total P, ET, ETincr, ETrain
ts_all=[] #all timeseries
for key in ['p','et','etrain','etincr']:
    nc=BASIN['main_data']['yearly'][key]
    var,name=open_nc(nc)
    Volume=var*area
    attrs=var.attrs
    attrs['units']='TCM/year'
    Volume.assign_attrs(attrs)
    ts=Volume.sum(dim=['latitude','longitude']).to_dataframe()
    ts.index=[y.year for y in ts.index]
    ts_all.append(ts)
    
### Calculate ET consumptions by Land Use Categories
LU,_=open_nc(BASIN['main_data']['yearly']['lu'])
ETg,_=open_nc(BASIN['main_data']['yearly']['etrain'])
ETb,_=open_nc(BASIN['main_data']['yearly']['etincr'])


### Different year date to year 
LU=LU.groupby('time.year').first(skipna=False)
ETb=area*ETb.groupby('time.year').first(skipna=False)
ETg=area*ETg.groupby('time.year').first(skipna=False)
### average per LU
from WA.average_by_LU import Total_perLU
ts_ETb=Total_perLU(ETb,LU)
ts_ETg=Total_perLU(ETg,LU)

ts_all.append(ts_ETb)
ts_all.append(ts_ETg)

# Read input time-series

In [5]:
### Read yearly dS from GRACE
df_dS_y=pd.read_csv(BASIN['input_ts']['dS_yearly'][0],sep=';',index_col=0)
Area_skm=np.nansum(area)
df_grace_ds=df_dS_y*Area_skm
df_grace_ds.index=[int(y[0:4]) for y in df_grace_ds.index]
df_grace_ds=df_grace_ds.rename(columns={'TWS Change [mm/year]':'dS_GRACE'})
ts_all.append(df_grace_ds)

### Read monthly Qout
for key in ['Qoutlet','Qswout','Qgwout']:
    if BASIN['input_ts'][key] is None:
        Q=ts_all[0]*0
        Q.columns=[key]
        ts_all.append(Q)
    else:        
        Q_m=pd.read_csv(BASIN['input_ts'][key],sep=';',index_col=0,skiprows=1)
        Q_m.index=[datetime.datetime.strptime(y,'%d/%m/%Y %H:%M') for y in Q_m.index]
        Q_m=Q_m.replace(-9999,np.nan)
        Q_y=Q_m.resample('A-{0}'.format(BASIN['end_month'])).sum() #mean()
        Q_y.index=[y.year for y in Q_y.index]
        ##
        df_Q_y=Q_y.where(Q_y.days>=365) ##remove years with missing data
        df_Q_y=df_Q_y['km3/month']*1000000 ##convert to TCM=km2*mm
        df_Q_y=df_Q_y.dropna()
        df_Q_y.name=key
        Q=df_Q_y.to_frame()
        ts_all.append(Q)

In [6]:
for i in range(len(ts_all)):
    if i ==0:
        df=ts_all[i]        
    else:
        df=pd.merge(df,ts_all[i],left_index=True,right_index=True,how='inner')
        
df['dS_WB']=df['Precipitation']-df['Actual Evapotranspiration']-df['Qoutlet']-df['Qswout']-df['Qgwout']
df['dS_Error']=df['dS_GRACE']-df['dS_WB']

OUT_CSV=os.path.join(BASIN['Dir'],'df_all.csv')
df.to_csv(OUT_CSV,sep=';')
BASIN['main_data']['df_all']=OUT_CSV
pickle_out(BASIN)

'D:\\Github\\WAPORWA\\data\\Litani\\Main\\Info_20200717_16h38.pickle'

# Calculate Yearly Sheet 1

In [7]:
#Get sheet1 csv template
WORKING_DIR=WA.__path__[0]
csv_template=os.path.join(WORKING_DIR,'csv','Sample_sheet1.csv')

# convert_unit=1000000 #from TCM to BCM
convert_unit=1000 #from TCM to MCM

df=pd.read_csv(BASIN['main_data']['df_all'],sep=';',index_col=0)
df

,Precipitation,Actual Evapotranspiration,Rainfall_ET_M,Incremental_ET_M,Incremental_ET_M-Protected Landuse,Incremental_ET_M-Utilized Landuse,Incremental_ET_M-Modified Landuse,Incremental_ET_M-Managed Water Use,Rainfall_ET_M-Protected Landuse,Rainfall_ET_M-Utilized Landuse,Rainfall_ET_M-Modified Landuse,Rainfall_ET_M-Managed Water Use,dS_GRACE,Qoutlet,Qswout,Qgwout,dS_WB,dS_Error
2010,1392988.40,1063525.10,521107.40,541917.25,0.0,246356.450,112911.695,182649.00,0.0,344671.90,63165.280,113270.22,-30489.372145,184059.216,0.000,0.0,145404.0340,-175893.406145
2011,1382399.60,976890.60,515424.88,461043.62,0.0,228115.600,84181.670,148746.36,0.0,319110.50,77588.740,118725.40,-57805.222377,195169.134,0.000,0.0,210339.8660,-268145.088377
2012,1782686.60,946824.50,520337.20,426056.22,0.0,185499.880,78611.470,161944.64,0.0,326458.97,67608.450,126269.79,44279.990926,362003.128,405574.560,0.0,68284.4370,-24004.446074
2013,1618594.20,1054580.10,577906.50,476255.10,0.0,216175.830,91722.210,168357.20,0.0,369136.34,79477.300,129292.98,-37281.509934,197105.964,381064.489,0.0,-14156.3280,-23125.181934
2014,995878.44,737642.44,512458.38,224739.16,0.0,88467.086,43438.610,92833.42,0.0,315382.94,68258.664,128816.67,55172.301448,63409.996,43817.925,0.0,151008.0790,-95835.777552
2015,1157414.40,820559.20,426739.75,393396.88,0.0,192192.280,64440.062,136764.48,0.0,265711.70,53788.930,107239.14,-27070.003737,168189.263,116738.293,0.0,51927.6315,-78997.635237
2016,1297710.10,767557.60,416656.75,350425.90,0.0,161996.980,67281.640,121147.28,0.0,269588.90,46539.453,100528.47,-42755.104300,91538.640,83957.234,0.0,354656.6260,-397411.730300


In [8]:
df=df/convert_unit
csv_folder=os.path.join(BASIN['Dir'],'data','csv')
if not os.path.exists(csv_folder):
    os.makedirs(csv_folder)

df_template=pd.read_csv(csv_template,sep=";")
csv_fhs=[]
for year in df.index:
    df_year=df_template.copy()
    df_year['VALUE']=0.0
    df_year.loc[0,'VALUE']=df.loc[year,'Precipitation']
#    if df.loc[year,'dS']>0:
#        df_year.loc[11,'VALUE']=df.loc[year,'dS_WB']
#    else:
#        df_year.loc[10,'VALUE']=abs(df.loc[year,'dS_WB'])
    df_year.loc[10,'VALUE']=-df.loc[year,'dS_WB']
    df_year.loc[12,'VALUE']=df.loc[year,'Rainfall_ET_M-Protected Landuse']
    df_year.loc[13,'VALUE']=df.loc[year,'Rainfall_ET_M-Utilized Landuse']
    df_year.loc[14,'VALUE']=df.loc[year,'Rainfall_ET_M-Modified Landuse']
    df_year.loc[15,'VALUE']=df.loc[year,'Rainfall_ET_M-Managed Water Use']
    df_year.loc[16,'VALUE']=df.loc[year,'Incremental_ET_M-Protected Landuse']
    df_year.loc[17,'VALUE']=df.loc[year,'Incremental_ET_M-Utilized Landuse']
    df_year.loc[18,'VALUE']=df.loc[year,'Incremental_ET_M-Modified Landuse']
    df_year.loc[19,'VALUE']=df.loc[year,'Incremental_ET_M-Managed Water Use']
    df_year.loc[20,'VALUE']=df.loc[year,'Incremental_ET_M-Managed Water Use']
    df_year.loc[21,'VALUE']=df.loc[year,'Incremental_ET_M']-df.loc[year,'Incremental_ET_M-Managed Water Use']
    df_year.loc[26,'VALUE']=df.loc[year,'Qswout']
    df_year.loc[22,'VALUE']=df.loc[year,'Qoutlet']
    outcsv=os.path.join(csv_folder,'Sheet1_{0}.csv'.format(int(year)))
    df_year.to_csv(outcsv,sep=";",index=False)
    csv_fhs.append(outcsv)